# **Webscraping of Toronto neighborhoods**
## Objectives

In this notebook we asemble the basic information about neighborhoods of Toronto, Ca.

This notebook is divided in two tasks:

1. Download from wikipedia all the neighborhoods from the city of Toronto, Ca.
2. Assign latlongs foreach neighborhood.

## Loading base packages

In [1]:
#!pip install bs4
#!pip install requestsanaco
#!pip3 install html5lib

# Loading base packages
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
import html5lib
from bs4 import BeautifulSoup # this module helps in web scrapping.
import requests  # this module helps us to download a web page

## 1. Wikipedia webscraping

Once found the convinient URL we retrive it trhought requests and parse it using beautifulsoup

In [2]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
data  = requests.get(url).text 
soup = BeautifulSoup(data,"html5lib")

The relevant information is withing the first table, so is enoguth to find "table" (or just to soup.table), thus, it's the first one.

In [3]:
table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

# print(table_contents)
df=pd.DataFrame(table_contents)

In [4]:
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

## 2. Requesting latlong

We will use geoencoder package with arcgis engine to get latlons. In order to do so we define the next function. It will be applied foreach postalcode element in the dataframe we just created.

In [10]:
def get_latlng(PostalCode):
    import geocoder
    g = geocoder.arcgis('{}, Toronto, Ontario'.format(PostalCode))
    return pd.Series([g.latlng[0], g.latlng[1]])

In [11]:
df[["Latitude","Longitude"]]=df.PostalCode.apply(get_latlng)

In [12]:
df

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.75245,-79.32991
1,M4A,North York,Victoria Village,43.73057,-79.31306
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65512,-79.36264
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.72327,-79.45042
4,M7A,Queen's Park,Ontario Provincial Government,43.66253,-79.39188
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.65319,-79.51113
99,M4Y,Downtown Toronto,Church and Wellesley,43.66659,-79.38133
100,M7Y,East Toronto Business,Enclave of M4L,43.64869,-79.38544
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.63278,-79.48945
